In [69]:
#get the datasets - 
# Imports
from urllib.request import urlopen

from metpy.io.gini import GiniFile
from siphon.catalog import TDSCatalog

# Grab the catalog and then the dataset for the most recent file
cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/satellite/SFC-T/SUPER-NATIONAL_1km/current/catalog.xml')
dataset_name = sorted(cat.datasets.keys())[-1]
dataset = cat.datasets[dataset_name]
supernat_latest= dataset.access_urls['OPENDAP']
dataset.access_urls

{'CdmRemote': 'http://thredds.ucar.edu/thredds/cdmremote/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170420_0600.gini',
 'HTTPServer': 'http://thredds.ucar.edu/thredds/fileServer/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170420_0600.gini',
 'ISO': 'http://thredds.ucar.edu/thredds/iso/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170420_0600.gini',
 'NCML': 'http://thredds.ucar.edu/thredds/ncml/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170420_0600.gini',
 'NetcdfSubset': 'http://thredds.ucar.edu/thredds/ncss/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170420_0600.gini',
 'OPENDAP': 'http://thredds.ucar.edu/thredds/dodsC/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170420_0600.gini',
 'UDDC': 'http://thredds.ucar.edu/thredds/uddc/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170420_0600.gini',
 'WCS': 'http://

In [31]:
#!/usr/bin/env python
# Read data from an opendap server
import netCDF4
# specify an url, - derived product
url = supernat_latest
# create a dataset object
dataset = netCDF4.Dataset(url)
 
# print the variables
dataset.variables

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
              float64 time(time)
                  long_name: time since base date
                  _CoordinateAxisType: Time
                  units: msecs since 1970-01-01T00:00:00Z
              unlimited dimensions: 
              current shape = (1,)
              filling off), ('x', <class 'netCDF4._netCDF4.Variable'>
              float64 x(x)
                  long_name: projection x coordinate
                  units: km
                  _CoordinateAxisType: GeoX
              unlimited dimensions: 
              current shape = (1536,)
              filling off), ('y', <class 'netCDF4._netCDF4.Variable'>
              float64 y(y)
                  long_name: projection y coordinate
                  units: km
                  _CoordinateAxisType: GeoY
              unlimited dimensions: 
              current shape = (1008,)
              filling off), ('Stereographic',
              <class 'netCDF4._netCDF4.Variab

In [32]:
#SFC_T - Surface Temperature
print(dataset.variables["SFC_T"][:])

[[[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]]


In [33]:
x = dataset.variables["x"]
#variable long name
x.getncattr("long_name")
# variable units
x.getncattr("units")

y = dataset.variables["y"]
#variable long name
y.getncattr("long_name")
# variable units
y.getncattr("units")

'km'

In [34]:
# get dimensions of x's and y's

nx = len(x)
nx
ny = len(y)
ny

1008

In [55]:
#get time
# get time
time = dataset.variables["time"]
tt=time[:][0]

In [25]:
# variable units
time.getncattr("units")

'msecs since 1970-01-01T00:00:00Z'

In [26]:
len(time)

1

In [ ]:
#only one timestamp as its a snapshot

In [36]:
# get surface temperature
# SFC_T
sfct_array = dataset.variables["SFC_T"]
#variable long name
sfct_array.getncattr("long_name")
# variable units
sfct_array.getncattr("units")
len(sfct_array)

1

In [37]:
sfct_array[:]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=int8)

In [65]:
sfct_array[0,1007,1000]

0

In [71]:
netcdf_latest= dataset.access_urls['NetcdfSubset']
from siphon.ncss import NCSS
ncss = NCSS(netcdf_latest)

In [72]:

ncss.variables

{'SFC_T'}

In [79]:
import numpy as np
from datetime import datetime, timedelta
# get current date and time
now = datetime.utcnow()
now = datetime(now.year, now.month, now.day, 6)

# define the time range we are interested in
start_time = now 
end_time = now

# build the query
query = ncss.query()
query.lonlat_point(-104.66, 39.85)
query.time_range(start_time, end_time)
query.variables('SFC_T')
query.accept('netcdf')

# what does the request url look like?
print(query)

var=SFC_T&time_start=2017-04-20T06%3A00%3A00&time_end=2017-04-20T06%3A00%3A00&longitude=-104.66&latitude=39.85&accept=netcdf


In [80]:
data = ncss.get_data(query)

In [81]:
print(list(data.variables))

['latitude', 'longitude', 'station_id', 'station_description', 'time', 'stationIndex', 'SFC_T']


In [82]:
time = [datetime.fromtimestamp(t) for t in data['time']]

In [83]:
time

[datetime.datetime(49270, 10, 7, 0, 0)]

In [84]:
data['SFC_T']

<class 'netCDF4._netCDF4.Variable'>
int8 SFC_T(obs)
    units: N/A
    long_name: Surface Skin Temperature
    coordinates: latitude longitude time
unlimited dimensions: obs
current shape = (1,)
filling off

In [88]:
data['SFC_T'][0]

0

In [89]:
station_id = data['station_id'][:].tostring()
print(station_id)

b'GridPoint'


In [90]:
sfct = data.variables['SFC_T'][:]

In [93]:
sfct[:][0]

0